# ADA 2020 - Project Milestone P2
## MOJO Team: Dorian Popović
## Publication: *Housing, Healt and Happiness*
This notebook attempts to repdroduce **Table 4** from the [Housing, Health and Happiness](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75) publication. In this study, researchers investigated the causal impact that housing improvement programs have on health and welfare. More specifically, they analyzed the effects of floor quality on the health of children and on the happiness and mental health of adults by using data from the **"Piso Firme Project"** (PFP) which replaced dirt floors by concrete cement flooring in Mexican households. 
<br>
In order for the replacement of dirt floors with cement provided by PFP to have beneficial effects on child and adult health, it was first necessary to ensure that PFP had a significant impact in terms of cement flooring in the households in which it was implemented. The authors first analysed the state level trends in cement floors and the results were in accordance with the hypothesis that PFP had an impact on cement flooring at the state level. Here we focus on estimating the program impact on cement flooring at the household level.

### **Estimated Program Impact on Cement Floors**
The aim here is to investigate the impact of PFP implementation on the presence of cement floors at the household level. For this we analyse the effect of the program in terms of coverage of cement floors in the treated households. Indeed it might be possible that households that did not beneficiate from the program replaced their dirt floors on their own, and this has to be controlled for. The following outcome indicators are examined:
* *Share of rooms with cement floors*
* *Kitchen has cement floor (dummy variable)*
* *Dining room has cement floor (dummy variable)*
* *Bathroom has cement floor (dummy variable)*
* *Household members sleep in rooms with a cement floor (dummy variable)*

PFP impact is estimated by regressing each of these dependent variables on a dummy variable indicating whether PFP was implemented in the household or not. For each dependent variable, 3 different linear regression models are estimated.
#### **Model 1**
The first model estimates the treatment effect (whether PFP was implemented in the household or not) directly by regressing each independent variable on a dummy variable that is equal to 1 if PFP was implemented and equal to 0 otherwise. No control variables were added.
#### **Model 2**
The second model adds the following demographic and health control variables:

**Demographic Control Variables** | **Health Control Variables** |
--- | --- |
*Household size* | *Animals* (dummy) |
*Number of rooms in the household* | *Animals can enter the house* (dummy) | 
*Education years of household head* | *Water connection outside* (dummy) |
*Education years of household spouse* | *Water connection inside* (dummy) |
*Age of household head* | *Electricity* (dummy) |
*Age of household spouse* | *Number of times the respondent washed hands before interview* |
*Group specific demographic (8)* | *Garbage collection service* (dummy) |

#### **Model 3**
The third model adds several control variables to Model 2 to control for benefits from other social programs:

**Social Program Benefits Control Variables** |
--- |
*Monetary transfer from social programs* |
*Milk supplement program* (dummy) |
*Any food program* (dummy) |
*Seguro Popular (health insurance)* (dummy) |

### **Table 4 Replication**

#### Imports and Constants Definition

In [1]:
import numpy as np
import pandas as pd
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import warnings
warnings.filterwarnings('ignore')

#***************************************************************
#                   DEFINITION OF CONSTANTS
#***************************************************************

# Data path constant
DATA_PATH = "./data/PisoFirme_AEJPol-20070024_household.dta"

# Dependent variables (dataset name)
DEP_VARS = ['S_shcementfloor', 
            'S_cementfloorkit', 
            'S_cementfloordin', 
            'S_cementfloorbat', 
            'S_cementfloorbed']

# Continuous control variables (dataset name)
CONT_CTRL_VARS = ['S_rooms',
                  'S_HHpeople',
                  'S_headeduc',
                  'S_spouseeduc',
                  'S_headage',
                  'S_spouseage',
                  'S_washhands',
                  'S_cashtransfers']

# Dummy NaN for continuous control variables (dataset name)
CONT_CTRL_VARS_NAN = [i + '_nan' for i in CONT_CTRL_VARS]

# Demographic control variables (dataset name)
DEMO_CTRL_VARS = ['S_dem'+str(i+1) for i in range(8)]

# Categorical control variables (dataset name)
DUMMY_CTRL_VARS = ['S_hasanimals',
                   'S_animalsinside',
                   'S_waterland',
                   'S_waterhouse',
                   'S_electricity',
                   'S_garbage',
                   'S_milkprogram',
                   'S_foodprogram',
                   'S_seguropopular']

# Dummy NaN for categorical control variables (dataset name)
DUMMY_CTRL_VARS_NAN = [i + '_nan' for i in DUMMY_CTRL_VARS]

# All control variables to generate NaN related dummies (dataset name) --> demographic variables not included
CTRL_VARS = CONT_CTRL_VARS + DUMMY_CTRL_VARS

# Variables for Model 1 linear regression (statsmodels name) --> single program dummy
MDL1_VARS = ['C(dpisofirme)']

# Variables for Model 2 linear regression (statsmodels name) --> add demographic and health control variables
MDL2_VARS = MDL1_VARS + CONT_CTRL_VARS[:7] + ['C('+i+')' for i in DUMMY_CTRL_VARS[:6]] \
                                           + ['C('+i+')' for i in CONT_CTRL_VARS_NAN[:7]] \
                                           + ['C('+i+')' for i in DUMMY_CTRL_VARS_NAN[:6]] \
                                           + DEMO_CTRL_VARS

# Variables for Model 3 linear regression (statsmodels name) --> add social program control variables
MDL3_VARS = MDL2_VARS + CONT_CTRL_VARS[-1:] + ['C('+i+')' for i in DUMMY_CTRL_VARS[-3:]] \
                                            + ['C('+i+')' for i in CONT_CTRL_VARS_NAN[-1:]] \
                                            + ['C('+i+')' for i in DUMMY_CTRL_VARS_NAN[-3:]] \

# Model variables without S_rooms for discussion part (statsmodels name)
MDL2_VARS_NOROOMS = [x for x in MDL2_VARS if x != 'S_rooms' and x != 'C(S_rooms_nan)']
MDL3_VARS_NOROOMS = [x for x in MDL3_VARS if x != 'S_rooms' and x != 'C(S_rooms_nan)']

# Names for table rows
ROWS = ['Share of rooms with cement floors',
        'Cement floor in kitchen',
        'Cement floor in dining room',
        'Cement floor in bathroom',
        'Cement floor in bedroom']

# Columns for the control group table
CG_COLUMNS = pd.MultiIndex.from_product([['Control Group'], ['Mean','Standard Deviation']])

# Program dummy name in statsmodels coefficients output
PROGRAMM_DUMMY = 'C(dpisofirme)[T.1.0]'

#### Functions Definition

In [2]:
#***************************************************************
#                   DEFINITION OF FUNCTIONS
#***************************************************************

# Function to calculate an entire model and store results in a dataframe

def calculate_model(df, which, var, data):
    # Input:
    # -- df, which: arguments for the add_model function (see below)
    # -- var: list of independent variables of the model
    # -- data: pandas dataframe with the data
    # Output:
    # -- mdl: the dataframe to which the desired model was added and in which the results are summarized
    
    # Add empty model to the previous dataframe
    mdl = add_model(df, which)
    
    # Compute regression results for each dependent variable and store them
    for i in range(len((DEP_VARS))):
        # Regress dependent variale i on variable(s) var
        ols = smf.ols(DEP_VARS[i] +'~' + '+'.join(var), data=data).fit(cov_type='cluster', cov_kwds={'groups':data['idcluster']})
        # Access the coefficient on the program dummy and store it
        mdl.loc[ROWS[i], (which, 'Coef.')] = ols.params[PROGRAMM_DUMMY]
        # Access the clustered standard error on the program dummy and store it --> display statistical significance based on the p-value
        mdl.loc[ROWS[i], (which, 'St. Err.')] = '['+str(ols.bse[PROGRAMM_DUMMY].round(3))+']***' if ols.pvalues['C(dpisofirme)[T.1.0]'] < 0.01 else ols.bse[PROGRAMM_DUMMY]
        # Compute ratio and store it
        mdl.loc[ROWS[i], (which, 'Ratio')] = 100*ols.params[PROGRAMM_DUMMY]/mdl.loc[ROWS[i], ('Control Group', 'Mean')]
    
    return mdl

# Function to add an empty model to the table
def add_model(df, which='No model'):
    # Input:
    # -- df: a pandas dataframe to which the empty model is added
    # -- which: a string for which model to add
    # Output:
    # -- added_model: the dataframe to which the desired empty model was added
    
    # Add a multi-index to the dataframe table for the specific model
    added_model = df.join(pd.DataFrame(np.zeros((5,3)),
                      columns=pd.MultiIndex.from_product([[which], ['Coef.','St. Err.','Ratio']]),
                     index=control_group.index))
    return added_model

### **1. Data Preprocessing**
#### Loading the Dataset
The `PisoFirme_AEJPol-20070024_household.dta` contains the results from the Piso Firme Project survey that were used in the publication to obtain the results. The dataset is loaded with the `read_stata` function and stored in a dataframe named **data**. In the raw dataset, not all observations have complete geographical information. They have NaN values at the **idcluster** feature which represents the *ID Census Block*. These observations are dropped. Additionally, the **idcluster** feature will be used as the clustering feature to report clustered standard errors (**136** clusters in total).

In [3]:
# Load dataset
data = pd.read_stata(DATA_PATH)
# Drop households whose geographical informations is not complete (NaN)
data = data[data['idcluster'].notna()]
# Make a copy for future use
data_copy = data.copy()

data

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cesds,S_pss,S_instcement,S_instsanita,S_restsanita,S_constceili,S_restowalls,S_improveany,S_logrent,S_logsell
0,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,14.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.903487
1,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,17.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,5.298317,9.615806
2,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,6.214608,10.819778
3,0.0,70000537.0,-103.503670,25.583067,7.0,47,0.300000,0.036629,0.555554,819.0,...,20.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,11.385092,11.918390
4,0.0,70000537.0,-103.503670,25.583067,7.0,47,0.300000,0.036629,0.555554,819.0,...,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.703783,10.819778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,1.0,353150000.0,-103.399841,25.501871,35.0,40,0.538462,0.100774,0.759924,454.0,...,19.0,20.0,1.0,0.0,0.0,1.0,0.0,1.0,5.298317,9.615806
2751,1.0,353150000.0,-103.399841,25.501871,35.0,40,0.538462,0.100774,0.759924,454.0,...,9.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,5.991465,10.819778
2752,1.0,353150000.0,-103.399841,25.501871,35.0,35,0.538462,0.100774,0.759924,454.0,...,12.0,19.0,1.0,0.0,0.0,0.0,0.0,0.0,5.991465,9.210340
2753,1.0,353150000.0,-103.399841,25.501871,35.0,34,0.538462,0.100774,0.759924,454.0,...,6.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,6.396930,11.918390


In [4]:
print('Treatment values counts in the dataset:')
print(data['dpisofirme'].value_counts())
print('Number of clusters based on Census Block ID:')
print(data['idcluster'].nunique())

Treatment values counts in the dataset:
0.0    1393
1.0    1362
Name: dpisofirme, dtype: int64
Number of clusters based on Census Block ID:
136


The dataset contains **2'755** household observations described by **78** features. In total **1'393** households did not receive PFP benefits and **1'362** did. The number of clusters based on the Census block ID is **136**. These results agree with the values that are reported in [Table 1](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75). Next, we deal with missing values.
#### Dealing with Missing Values
Now that we have information about the households of interest, we need to deal with the missing values. The authors stated that  "*Missing values in
covariates were imputed with zero, and a corresponding dummy variable was then added to the regressions*". The idea is to create a **dummy variable** that is equal to **1** if the feature is **NaN** and **0** otherwise for each control variable before imputing the misisng values with 0. However when examining how many missing values there are for each control variable:

In [5]:
# Display the total count of NaN values for each control variable
data[DEMO_CTRL_VARS + CTRL_VARS].isna().sum()

S_dem1               1
S_dem2               1
S_dem3               1
S_dem4               1
S_dem5               1
S_dem6               1
S_dem7               1
S_dem8               1
S_rooms              0
S_HHpeople           0
S_headeduc           4
S_spouseeduc       337
S_headage            0
S_spouseage          0
S_washhands          0
S_cashtransfers      2
S_hasanimals         0
S_animalsinside      0
S_waterland          0
S_waterhouse         0
S_electricity        0
S_garbage            0
S_milkprogram        0
S_foodprogram        0
S_seguropopular      0
dtype: int64

We can see that here is a different number of missing values for each control variable except for the demographic control variables **S_dem** that all have a single missing value. This missing value happens at the same row for each one of them:

In [6]:
# Create an array that returns the index where an entire row is NaN
np.nonzero(data[DEMO_CTRL_VARS].isnull().apply(lambda x: all(x), axis=1).values)

(array([2127]),)

For every demographic control variable, only the **2127th** row has a missing value. Creating a dummy variable for them would be useless as it would always give the same one and might lead to incorrect regressions results. Therefore, a corresponding dummy variable is created for each control variable except the **S_dem** ones. Finally, all **NaN** values are imputed with **0**.

In [7]:
# Generate dummies for NaN values for all control variables except S_dem
data = pd.concat([data, pd.get_dummies(data[CTRL_VARS], columns=CTRL_VARS, dummy_na=True)[CONT_CTRL_VARS_NAN + DUMMY_CTRL_VARS_NAN]], axis=1) 
# Impute all NaN values with 0
data = data.fillna(0)
data

,dpisofirme,idcluster,coord_x,coord_y,idmun,idmza,C_blocksdirtfloor,C_HHdirtfloor,C_child05,C_households,...,S_cashtransfers_nan,S_hasanimals_nan,S_animalsinside_nan,S_waterland_nan,S_waterhouse_nan,S_electricity_nan,S_garbage_nan,S_milkprogram_nan,S_foodprogram_nan,S_seguropopular_nan
0,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,70000537.0,-103.503670,25.583067,7.0,40,0.300000,0.036629,0.555554,819.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,70000537.0,-103.503670,25.583067,7.0,47,0.300000,0.036629,0.555554,819.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,70000537.0,-103.503670,25.583067,7.0,47,0.300000,0.036629,0.555554,819.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2750,1.0,353150000.0,-103.399841,25.501871,35.0,40,0.538462,0.100774,0.759924,454.0,...,0,0,0,0,0,0,0,0,0,0
2751,1.0,353150000.0,-103.399841,25.501871,35.0,40,0.538462,0.100774,0.759924,454.0,...,0,0,0,0,0,0,0,0,0,0
2752,1.0,353150000.0,-103.399841,25.501871,35.0,35,0.538462,0.100774,0.759924,454.0,...,0,0,0,0,0,0,0,0,0,0
2753,1.0,353150000.0,-103.399841,25.501871,35.0,34,0.538462,0.100774,0.759924,454.0,...,0,0,0,0,0,0,0,0,0,0


### **2. Calculating Control Group Means and Standard Deviations**
The first replication step is to calculate the **mean** and the **standard deviation** of each outcome variable in the control households (those that did not receive PFP benefits). This is done in order to assess the size of the coefficients that will be estimated subsequently. We first create an empty dataframe called **control_group**. The results are calculated by grouping values by `dpisofirme` **= 0** and calculating the mean and standard deviation for each dependent variable.

In [8]:
# Create empty dataframe for the control group results
control_group = pd.DataFrame(index=ROWS, columns=CG_COLUMNS)
control_group.index.name = 'Dependent Variable'

# Fill in  columns with mean and standard deviation values for each outcome variable
control_group['Control Group','Mean'] = data.groupby('dpisofirme', as_index=True)[DEP_VARS].mean().loc[0.0,:].values
control_group['Control Group','Standard Deviation'] = data.groupby('dpisofirme', as_index=True)[DEP_VARS].std().loc[0.0,:].values
control_group.round(3)

Control Group                   
                                           Mean Standard Deviation
Dependent Variable                                                
Share of rooms with cement floors         0.728              0.363
Cement floor in kitchen                   0.671              0.470
Cement floor in dining room               0.709              0.455
Cement floor in bathroom                  0.803              0.398
Cement floor in bedroom                   0.668              0.471

The results obtained for the control group mean and standard deviation for each dependent variable of interest are identical to the results reported in the first column of [Table 4](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75). We can see that around **73%** of the control households rooms had cement floor in 2005. In 2000 this share was equal to **33%** (not shown here) meaning that there was a **40-percentage point** increase in cement flooring from 2000 to 2005 even without PFP.

### **3. Assessing Model 1**
Model 1 regresses each independent variable on a dummy variable that is equal to 1 if PFP was implemented and equal to 0 otherwise. No control variables were added. The following results are reported:
* **Coef.**: the estimated linear regression coefficient on the program dummy
* **St. Err**: clustered standard error at census-level block
* **Ratio**: 100*coefficient/control mean
* ***: significantly different from 0 at a 1 percent level

In [9]:
# Adding Model 1 to the table and fill values with results from linear regression
model1 = calculate_model(df=control_group, which='Model 1', var=MDL1_VARS, data=data)
model1.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Share of rooms with cement floors         0.728              0.363   0.202   
Cement floor in kitchen                   0.671              0.470   0.255   
Cement floor in dining room               0.709              0.455   0.210   
Cement floor in bathroom                  0.803              0.398   0.105   
Cement floor in bedroom                   0.668              0.471   0.238   

                                                       
                                     St. Err.   Ratio  
Dependent Variable                                     
Share of rooms with cement floors  [0.021]***  27.746  
Cement floor in kitchen            [0.025]***  37.936  
Cement floor in dining room        [0.026]***  29.633  
Cement floor in bathroom           [0.022]***  13.071  
Cement floor in bedroom             [0.02]***  35.598

The results obtained for Model 1 are identical to those reported in [Table 4](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75). We can see that depsite the large secular increase in cement floor observed in the previous section for the control group, the PFP has a large positive effect on cement floor installation for all indicators studied. PFP brought an increment of approximately **28%** in the *share of rooms with cement floors*. The effects are stronger in **kitchens** and **bedrooms** in treated households. However, this linear regression model does not include any control variable and might suffer from omitted variable bias.

### **4. Assessing Model 2**
Model 2 adds several demographic and health control variables to control for potential omitted variable bias. The control variables are summarized in the **Model 2** part of the **Estimated Program Impact on Cement Floors** section.

In [10]:
# Adding Model 2 to the table and fill values with results from linear regression
model2 = calculate_model(df=model1, which='Model 2', var=MDL2_VARS, data=data)
model2.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Share of rooms with cement floors         0.728              0.363   0.202   
Cement floor in kitchen                   0.671              0.470   0.255   
Cement floor in dining room               0.709              0.455   0.210   
Cement floor in bathroom                  0.803              0.398   0.105   
Cement floor in bedroom                   0.668              0.471   0.238   

                                                      Model 2              \
                                     St. Err.   Ratio   Coef.    St. Err.   
Dependent Variable                                                          
Share of rooms with cement floors  [0.021]***  27.746   0.203  [0.019]***   
Cement floor in kitchen            [0.025]***  37.936   0.255  [0.022]***   
Cement floor in dining room        [0.026]***  29.633   0.212  [0.024]***   
Cement floor in bathroom           [0.022]***  13.071   0.108  [0.018]***   
Cement floor in bedroom             [0.02]***  35.598   0.243   [0.02]***   

                                           
                                    Ratio  
Dependent Variable                         
Share of rooms with cement floors  27.892  
Cement floor in kitchen            37.944  
Cement floor in dining room        29.883  
Cement floor in bathroom           13.488  
Cement floor in bedroom            36.388

This time the coefficients as well as the ratio are systematically slightly lower than those reported in [Table 4](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75). However the effect of PFP is still statistically different from 0 at a 1 percent level. This difference might be due to errors in the model itself or incorrect description in the paper but it cannot be attributed to differences in rouding only.

### **5. Assessing Model 3**
Finally, Model 3 adds several control variables to Model 2 to control for benefits from other social programs. These variables are summarized in the **Model 3** part of the **Estimated Program Impact on Cement Floors section**.

In [11]:
# Adding Model 3 to the table and fill values with results from linear regression
model3 = calculate_model(df=model2, which='Model 3', var=MDL3_VARS, data=data)
model3.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Share of rooms with cement floors         0.728              0.363   0.202   
Cement floor in kitchen                   0.671              0.470   0.255   
Cement floor in dining room               0.709              0.455   0.210   
Cement floor in bathroom                  0.803              0.398   0.105   
Cement floor in bedroom                   0.668              0.471   0.238   

                                                      Model 2              \
                                     St. Err.   Ratio   Coef.    St. Err.   
Dependent Variable                                                          
Share of rooms with cement floors  [0.021]***  27.746   0.203  [0.019]***   
Cement floor in kitchen            [0.025]***  37.936   0.255  [0.022]***   
Cement floor in dining room        [0.026]***  29.633   0.212  [0.024]***   
Cement floor in bathroom           [0.022]***  13.071   0.108  [0.018]***   
Cement floor in bedroom             [0.02]***  35.598   0.243   [0.02]***   

                                          Model 3                      
                                    Ratio   Coef.    St. Err.   Ratio  
Dependent Variable                                                     
Share of rooms with cement floors  27.892   0.206  [0.019]***  28.267  
Cement floor in kitchen            37.944   0.260  [0.022]***  38.692  
Cement floor in dining room        29.883   0.216  [0.024]***  30.497  
Cement floor in bathroom           13.488   0.112  [0.018]***  13.987  
Cement floor in bedroom            36.388   0.243   [0.02]***  36.356

Again, the coefficients and ratio that are found are slightly lower than those reported in [Table 4](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75) but the effects remain statistically different from 0 at a 1 percent level.

### **Conclusion**
Overall, the results tell us that PFP succeeded in installing cement floors in the treated households for almost all of the household floor space and they agree with the results reported in the [Housing, Health and Happiness](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75) study paper despite some small differences in **Model 2** and **Model 3**. These differences will be discussed in the next section.
### **Discussion**
As mentionned previously the replication results agree with the study results but there are some small discrepancies regarding the coefficients in **Model 2** and **Model 3** that cannot be explained by differences in rounding only. These discrepancies also cannot come from differences in the dataset as **Control Group** and **Model 1** show identical results and the same dataset is used throughout this replication. Therefore the only explanations are either an error in the model itself (for example false control variables used) or an error in the description of the model in the paper. When analysing the replication code `PisoFirme_AEJPol-20070024_STATA-replication-code.do` that is provided together with the dataset, even though the variable **S_rooms** that represents the *number of rooms* in the household is mentionned to be controlled for in both models in the paper, it is actually not controlled for in the replication code. Below are reported the results for **Model 2** and **Model 3** when the variable **S_rooms** is  removed.

In [12]:
# Adding Model 2 and 3 without the S_rooms control variable
model2_norooms = calculate_model(df=model3, which='Model 2 (no S_rooms)', var=MDL2_VARS_NOROOMS, data=data)
model3_norooms = calculate_model(df=model2_norooms, which='Model 3 (no S_rooms)', var=MDL3_VARS_NOROOMS, data=data)
model3_norooms.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Share of rooms with cement floors         0.728              0.363   0.202   
Cement floor in kitchen                   0.671              0.470   0.255   
Cement floor in dining room               0.709              0.455   0.210   
Cement floor in bathroom                  0.803              0.398   0.105   
Cement floor in bedroom                   0.668              0.471   0.238   

                                                      Model 2              \
                                     St. Err.   Ratio   Coef.    St. Err.   
Dependent Variable                                                          
Share of rooms with cement floors  [0.021]***  27.746   0.203  [0.019]***   
Cement floor in kitchen            [0.025]***  37.936   0.255  [0.022]***   
Cement floor in dining room        [0.026]***  29.633   0.212  [0.024]***   
Cement floor in bathroom           [0.022]***  13.071   0.108  [0.018]***   
Cement floor in bedroom             [0.02]***  35.598   0.243   [0.02]***   

                                          Model 3                      \
                                    Ratio   Coef.    St. Err.   Ratio   
Dependent Variable                                                      
Share of rooms with cement floors  27.892   0.206  [0.019]***  28.267   
Cement floor in kitchen            37.944   0.260  [0.022]***  38.692   
Cement floor in dining room        29.883   0.216  [0.024]***  30.497   
Cement floor in bathroom           13.488   0.112  [0.018]***  13.987   
Cement floor in bedroom            36.388   0.243   [0.02]***  36.356   

                                  Model 2 (no S_rooms)                      \
                                                 Coef.    St. Err.   Ratio   
Dependent Variable                                                           
Share of rooms with cement floors                0.208  [0.019]***  28.512   
Cement floor in kitchen                          0.260  [0.023]***  38.708   
Cement floor in dining room                      0.217  [0.025]***  30.588   
Cement floor in bathroom                         0.113  [0.018]***  14.043   
Cement floor in bedroom                          0.245  [0.021]***  36.735   

                                  Model 3 (no S_rooms)                      
                                                 Coef.    St. Err.   Ratio  
Dependent Variable                                                          
Share of rooms with cement floors                0.210  [0.019]***  28.876  
Cement floor in kitchen                          0.265  [0.023]***  39.440  
Cement floor in dining room                      0.221  [0.025]***  31.189  
Cement floor in bathroom                         0.117  [0.018]***  14.536  
Cement floor in bedroom                          0.245   [0.02]***  36.695

In [13]:
model3_norooms.to_csv("./data/cement_regression.csv", index=False)

Note that once the **S_rooms** control variable was removed from both models, the obtained results are identical. It is therefore most likely that researchers forgot to control for **S_rooms** in their model even though they mentionned it in the paper. Despite these small differences, the effects are still the same and remain statistically significant in both cases. Overall, this pure replication analysis concludes that results in **Table 4** from the [Housing, Health and Happiness](https://www.aeaweb.org/articles?id=10.1257/pol.1.1.75) publication can be replicated and that there might be some minor errors that do not alter the original author’s main conclusions.

### **Table 6 Replication**

#### **Variable re-definition**

In [14]:
# Dependent variables (dataset name)
DEP_VARS = ['S_satisfloor', 
            'S_satishouse', 
            'S_satislife', 
            'S_cesds', 
            'S_pss']

# Names for table rows
ROWS = ['Satisfaction with floor quality',
        'Satisfaction with house quality',
        'Satisfaction with quality of life',
        'Depression scale (CES-D scale)',
        'Perceived stress scale (PSS)']

#### **New Functions**

In [15]:
def compute_single_feature(data, feature, index):
    # Create empty dataframe for the control group results
    cg = pd.DataFrame(index=index, columns=CG_COLUMNS)
    cg.index.name = 'Dependent Variable'

    # Fill in  columns with mean and standard deviation values for each outcome variable
    cg['Control Group','Mean'] = data.groupby('dpisofirme', as_index=True)[feature].mean().loc[0.0]
    cg['Control Group','Standard Deviation'] = data.groupby('dpisofirme', as_index=True)[feature].std().loc[0.0]
    cg.round(3)

    # Adding Model 1 to the table and fill values with results from linear regression
    mdl1 = calculate_model(df=cg, which='Model 1', var=MDL1_VARS, data=data)
    mdl2 = calculate_model(df=mdl1, which='Model 2 (no S_rooms)', var=MDL2_VARS_NOROOMS, data=data)
    mdl3 = calculate_model(df=mdl2, which='Model 3 (no S_rooms)', var=MDL3_VARS_NOROOMS, data=data)
    
    return mdl3.round(3).loc[index,:]

In [16]:
# Create empty dataframe for the control group results
control_group = pd.DataFrame(index=ROWS, columns=CG_COLUMNS)
control_group.index.name = 'Dependent Variable'

# Fill in  columns with mean and standard deviation values for each outcome variable
control_group['Control Group','Mean'] = data.groupby('dpisofirme', as_index=True)[DEP_VARS].mean().loc[0.0,:].values
control_group['Control Group','Standard Deviation'] = data.groupby('dpisofirme', as_index=True)[DEP_VARS].std().loc[0.0,:].values
control_group.round(3)

Control Group                   
                                           Mean Standard Deviation
Dependent Variable                                                
Satisfaction with floor quality        0.511000              0.500
Satisfaction with house quality        0.605000              0.489
Satisfaction with quality of life      0.601000              0.490
Depression scale (CES-D scale)        18.466000              9.450
Perceived stress scale (PSS)          16.443001              6.983

In [17]:
# Adding Model 1 to the table and fill values with results from linear regression
model1 = calculate_model(df=control_group, which='Model 1', var=MDL1_VARS, data=data)
model1.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Satisfaction with floor quality        0.511000              0.500   0.219   
Satisfaction with house quality        0.605000              0.489   0.092   
Satisfaction with quality of life      0.601000              0.490   0.112   
Depression scale (CES-D scale)        18.466000              9.450  -2.344   
Perceived stress scale (PSS)          16.443001              6.983  -1.712   

                                                       
                                     St. Err.   Ratio  
Dependent Variable                                     
Satisfaction with floor quality    [0.023]***  42.784  
Satisfaction with house quality    [0.021]***  15.136  
Satisfaction with quality of life  [0.022]***  18.650  
Depression scale (CES-D scale)     [0.603]*** -12.694  
Perceived stress scale (PSS)       [0.421]*** -10.414

In [18]:
# Adding Model 2 to the table and fill values with results from linear regression
model2_norooms = calculate_model(df=model1, which='Model 2 (no S_rooms)', var=MDL2_VARS_NOROOMS, data=data)
model2_norooms.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Satisfaction with floor quality        0.511000              0.500   0.219   
Satisfaction with house quality        0.605000              0.489   0.092   
Satisfaction with quality of life      0.601000              0.490   0.112   
Depression scale (CES-D scale)        18.466000              9.450  -2.344   
Perceived stress scale (PSS)          16.443001              6.983  -1.712   

                                                      Model 2 (no S_rooms)  \
                                     St. Err.   Ratio                Coef.   
Dependent Variable                                                           
Satisfaction with floor quality    [0.023]***  42.784                0.223   
Satisfaction with house quality    [0.021]***  15.136                0.087   
Satisfaction with quality of life  [0.022]***  18.650                0.112   
Depression scale (CES-D scale)     [0.603]*** -12.694               -2.449   
Perceived stress scale (PSS)       [0.421]*** -10.414               -1.733   

                                                       
                                     St. Err.   Ratio  
Dependent Variable                                     
Satisfaction with floor quality    [0.024]***  43.635  
Satisfaction with house quality    [0.021]***  14.369  
Satisfaction with quality of life  [0.021]***  18.557  
Depression scale (CES-D scale)     [0.559]*** -13.260  
Perceived stress scale (PSS)        [0.39]*** -10.537

In [19]:
model3_norooms = calculate_model(df=model2_norooms, which='Model 3 (no S_rooms)', var=MDL3_VARS_NOROOMS, data=data)
model3_norooms.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Satisfaction with floor quality        0.511000              0.500   0.219   
Satisfaction with house quality        0.605000              0.489   0.092   
Satisfaction with quality of life      0.601000              0.490   0.112   
Depression scale (CES-D scale)        18.466000              9.450  -2.344   
Perceived stress scale (PSS)          16.443001              6.983  -1.712   

                                                      Model 2 (no S_rooms)  \
                                     St. Err.   Ratio                Coef.   
Dependent Variable                                                           
Satisfaction with floor quality    [0.023]***  42.784                0.223   
Satisfaction with house quality    [0.021]***  15.136                0.087   
Satisfaction with quality of life  [0.022]***  18.650                0.112   
Depression scale (CES-D scale)     [0.603]*** -12.694               -2.449   
Perceived stress scale (PSS)       [0.421]*** -10.414               -1.733   

                                                      Model 3 (no S_rooms)  \
                                     St. Err.   Ratio                Coef.   
Dependent Variable                                                           
Satisfaction with floor quality    [0.024]***  43.635                0.222   
Satisfaction with house quality    [0.021]***  14.369                0.084   
Satisfaction with quality of life  [0.021]***  18.557                0.112   
Depression scale (CES-D scale)     [0.559]*** -13.260               -2.399   
Perceived stress scale (PSS)        [0.39]*** -10.537               -1.705   

                                                       
                                     St. Err.   Ratio  
Dependent Variable                                     
Satisfaction with floor quality    [0.026]***  43.421  
Satisfaction with house quality    [0.022]***  13.892  
Satisfaction with quality of life  [0.022]***  18.701  
Depression scale (CES-D scale)     [0.551]*** -12.993  
Perceived stress scale (PSS)       [0.388]*** -10.372

When using the same methods as previously, we can see that that **Satisfaction with floor quality**, **Satisfaction with house quality** and **Satisfaction with quality of life** are correctly reproduced. However, the last two row, **Depression scale (CES-D scale)** and **Perceived stress scale (PSS)** are not identical. The sign on the coefficient is the same (negative) as excpected but the values are not the same. Because the **control groupe mean** is not the same, the differences are probably due to a change in number of observations. Indeed if we look at **Table 1** from th study, the number of observations for these last 2 variables is not the same as for every other variable used in this notebook. In this next part we will create two separate dataframe and drop observation based on **NaN** values for the **depression** and **stress** features.

In [20]:
data_cesds = data_copy[data_copy['S_cesds'].notna()]
data_pss = data_copy[data_copy['S_pss'].notna()]

# Generate dummies for NaN values for all control variables except S_dem
data_cesds = pd.concat([data_cesds, pd.get_dummies(data_cesds[CTRL_VARS], columns=CTRL_VARS, dummy_na=True)[CONT_CTRL_VARS_NAN + DUMMY_CTRL_VARS_NAN]], axis=1) 
# Impute all NaN values with 0
data_cesds = data_cesds.fillna(0)

# Generate dummies for NaN values for all control variables except S_dem
data_pss = pd.concat([data_pss, pd.get_dummies(data_pss[CTRL_VARS], columns=CTRL_VARS, dummy_na=True)[CONT_CTRL_VARS_NAN + DUMMY_CTRL_VARS_NAN]], axis=1) 
# Impute all NaN values with 0
data_pss = data_pss.fillna(0)

In [21]:
stress = compute_single_feature(data=data_pss, feature='S_pss', index=[ROWS[-1]])
stress

Control Group                    Model 1  \
                                      Mean Standard Deviation   Coef.   
Dependent Variable                                                      
Perceived stress scale (PSS)        16.514              6.914  -1.751   

                                                 Model 2 (no S_rooms)  \
                                St. Err.   Ratio                Coef.   
Dependent Variable                                                      
Perceived stress scale (PSS)  [0.428]*** -10.603               -1.769   

                                                Model 3 (no S_rooms)  \
                                St. Err.  Ratio                Coef.   
Dependent Variable                                                     
Perceived stress scale (PSS)  [0.396]*** -10.71               -1.742   

                                                  
                                St. Err.   Ratio  
Dependent Variable                                
Perceived stress scale (PSS)  [0.396]*** -10.551

In [22]:
depression = compute_single_feature(data=data_cesds, feature='S_cesds', index=[ROWS[-2]])
depression

Control Group                    Model 1  \
                                        Mean Standard Deviation   Coef.   
Dependent Variable                                                        
Depression scale (CES-D scale)        18.532              9.402  -2.315   

                                                   Model 2 (no S_rooms)  \
                                  St. Err.   Ratio                Coef.   
Dependent Variable                                                        
Depression scale (CES-D scale)  [0.616]*** -12.493               -2.417   

                                                  Model 3 (no S_rooms)  \
                                 St. Err.   Ratio                Coef.   
Dependent Variable                                                       
Depression scale (CES-D scale)  [0.57]*** -13.043               -2.372   

                                                    
                                  St. Err.   Ratio  
Dependent Variable                                  
Depression scale (CES-D scale)  [0.562]*** -12.797

In [23]:
model3_norooms.loc[ROWS[-1],:] = stress.loc[ROWS[-1],:]
model3_norooms.loc[ROWS[-2],:] = depression.loc[ROWS[-2],:]
model3_norooms.round(3)

Control Group                    Model 1  \
                                           Mean Standard Deviation   Coef.   
Dependent Variable                                                           
Satisfaction with floor quality           0.511              0.500   0.219   
Satisfaction with house quality           0.605              0.489   0.092   
Satisfaction with quality of life         0.601              0.490   0.112   
Depression scale (CES-D scale)           18.532              9.402  -2.315   
Perceived stress scale (PSS)             16.514              6.914  -1.751   

                                                      Model 2 (no S_rooms)  \
                                     St. Err.   Ratio                Coef.   
Dependent Variable                                                           
Satisfaction with floor quality    [0.023]***  42.784                0.223   
Satisfaction with house quality    [0.021]***  15.136                0.087   
Satisfaction with quality of life  [0.022]***  18.650                0.112   
Depression scale (CES-D scale)     [0.616]*** -12.493               -2.417   
Perceived stress scale (PSS)       [0.428]*** -10.603               -1.769   

                                                      Model 3 (no S_rooms)  \
                                     St. Err.   Ratio                Coef.   
Dependent Variable                                                           
Satisfaction with floor quality    [0.024]***  43.635                0.222   
Satisfaction with house quality    [0.021]***  14.369                0.084   
Satisfaction with quality of life  [0.021]***  18.557                0.112   
Depression scale (CES-D scale)      [0.57]*** -13.043               -2.372   
Perceived stress scale (PSS)       [0.396]*** -10.710               -1.742   

                                                       
                                     St. Err.   Ratio  
Dependent Variable                                     
Satisfaction with floor quality    [0.026]***  43.421  
Satisfaction with house quality    [0.022]***  13.892  
Satisfaction with quality of life  [0.022]***  18.701  
Depression scale (CES-D scale)     [0.562]*** -12.797  
Perceived stress scale (PSS)       [0.396]*** -10.551

Now that the linear regressions have been computed with the right dataset, the results are identical.